In [ ]:
# Imports
import os
import getpass

import numpy as np

import nipype
import nibabel
import jinja2
import dipy.io.streamline as dis

import nipype.interfaces.io as nio
import dipy.io.streamline as dis
import os


from nipype.interfaces.base import BaseInterface
from nipype.interfaces.utility import IdentityInterface
from dipy.viz import window, actor

from nipype.interfaces.base import BaseInterfaceInputSpec, File, TraitedSpec, traits
from nipype import Node, Function, Workflow
from nipype.interfaces.io import BIDSDataGrabber
from nilearn.plotting import plot_roi
from nilearn import image
from bids.layout import BIDSLayout
from nipype.pipeline import engine as pe

from matplotlib import pyplot as plt
from os.path import join
from bids.tests import get_test_data_path

In [ ]:
user = getpass.getuser()

if user == 'ari':
    bids_ds_path = '/Volumes/SSD-Bercows/Joan/research/0_data/Sample_Multi-Modal_BIDS_dataset/ds-sample/'

if user == 'andres':
    bids_ds_path = '/media/sf_UbuntuShare/BIDS/DS001_BIDS_andres/'

deriv_cmp_dir = join(bids_ds_path, "derivatives", "cmp")
deriv_nipype_dir = join(bids_ds_path, "nipype", "sub-01", "ses-01")

In [ ]:
class TractographyPlotInputSpec(BaseInterfaceInputSpec):
    trk_file = traits.List(exists=True, mandatory=True, desc='trk file')

    
class TractographyPlotOutputSpec(TraitedSpec):
    fig_file = traits.List(desc='png tractography')
    
    

class TractographyPlot(BaseInterface):
    input_spec = TractographyPlotInputSpec
    output_spec = TractographyPlotOutputSpec
    
    def _run_interface(self, runtime):

        # Call our python code here:
        self.fig_file = []
        self.fig_file.append(self._tractography_plot(
            self.inputs.trk_file[0]
        ))

        # And we are done
        return runtime
    
    def _tractography_plot(self, trk_path):

        trk_image = dis.load_trk(trk_path,trk_path)

        n_spag = len(trk_image.streamlines)
        ids = np.random.permutation(n_spag)[:int(n_spag*0.01)]
        streamlines = trk_image.streamlines[ids]

        scene = window.Scene()

        stream_actor = actor.line(streamlines)

        scene.set_camera(position=(-176.42, 118.52, 128.20),
                         focal_point=(113.30, 128.31, 76.56),
                         view_up=(0.18, 0.00, 0.98))

        scene.add(stream_actor)
        
        output_file = bids_ds_path + 'derivatives/nipype/sub-01/tractography.png'

        # Uncomment the line below to show to display the window
        # window.show(scene, size=(600, 600), reset_camera=False)
        window.record(scene, out_path=output_file, size=(2400, 2400))
        
        
        return output_file
    
    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs['fig_file'] = self.fig_file
        return outputs

In [ ]:
class ReportInputSpec(BaseInterfaceInputSpec):
    fig_file = traits.List(desc='png overlay')

    
class ReportOutputSpec(TraitedSpec):
    report_file = File(desc='HTML report file')
    

class Report(BaseInterface):
    input_spec = ReportInputSpec
    #output_spec = ReportOutputSpec
    
    def _run_interface(self, runtime):
        
        # Call our python code here:
        self.fig_file = self._generate_html(
            self.inputs.fig_file
        )
        
        # And we are done
        return runtime
    
    def _generate_html(self, fig_file):
        title = 'Hi this is my title'
        outputfile = '/home/andres/Desktop/BIDS/automatedHTML/report_segmentation.html'
        
        my_images = []
        for image in fig_file:
            my_images.append([image, image.split('/')[-1].split('.')[-2]])
            
        subs = jinja2.Environment(
                      loader=jinja2.FileSystemLoader('/home/andres/Desktop/BIDS/automatedHTML/templates')
                      ).get_template('template_segmentation.html').render(title=title,mydata=my_images)
        # lets write the substitution to a file
        with open(outputfile,'w') as f: f.write(subs)

In [ ]:
bg = Node(BIDSDataGrabber(index_derivatives=True), name='bids-grabber')
bg.inputs.base_dir = bids_ds_path

bg.inputs.subject = '01'

bg.inputs.output_query = {'TRK': {'scope': 'Connectome Mapper',
                                  'model': 'CSD',
                                  'desc': 'DET',
                                  'sufix': 'tractogram',
                                  'extensions': ['trk']}
                         }

In [ ]:
overlay = Node(TractographyPlot(), name='tractography_fig')
report = Node(Report(), name='report')
datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory = deriv_nipype_dir

In [ ]:
workflow = Workflow("tractography_workflow")

workflow.connect([(bg, overlay,
                   [('TRK','trk_file')]
                  )])
workflow.connect(overlay, 'fig_file', datasink, 'fig_input')
workflow.connect(overlay, 'fig_file', report, 'fig_file')

workflow.run()